In [1]:

from functools import partial
import pandas as pd
import time
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestRegressor as rfr
import shap
import xgboost as xgb
import numpy as np

### Models

In [2]:
from BioML.features import selection
from pycaret.datasets import get_data
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path

In [3]:
data = get_data('diabetes')
new_data = data.drop("Class variable", axis=1)
labels = data["Class variable"]

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
ester_label = pd.read_csv("../data/esterase_labels.csv", index_col=0)
ester = "-../data/esterase_features.xlsx"

In [1]:
{}.get("hello", "possum_features")

'possum_features'

## Classification

In [5]:
import BioML.models.classification as classification
from collections import defaultdict
from functools import partial
import pandas as pd
import numpy as np

In [6]:
plot = ("learning", "confusion_matrix", "class_report")

In [7]:
excel = pd.read_excel("../data/esterase_features.xlsx", index_col=0, sheet_name="ch2_20")
label = pd.read_csv("../data/esterase_labels.csv", index_col=0)
l = pd.Series(label["label1"])

In [8]:
data = classification.DataParser("../data/esterase_features.xlsx", "../data/esterase_labels.csv", sheets="ch2_20")
experiment = classification.PycaretInterface("classification", 250, budget_time=20, best_model=3, 
                                  output_path="classification_training", optimize="MCC")
classifier = classification.Classifier(optimize="MCC", drop=())
training = classification.Trainer(experiment, classifier, 5, 50)


27-07-2024 11:57:36 INFO ------------------------------------------------------------------------------
27-07-2024 11:57:36 INFO PycaretInterface parameters
27-07-2024 11:57:36 INFO Seed: 250
27-07-2024 11:57:36 INFO Budget time: 20
27-07-2024 11:57:36 INFO The number of models to select: 3
27-07-2024 11:57:36 INFO Output path: classification_training
27-07-2024 11:57:36 INFO ----------------Trainer inputs-------------------------
27-07-2024 11:57:36 INFO Number of kfolds: 5
27-07-2024 11:57:36 INFO Number of retuning iterations: 50
27-07-2024 11:57:36 INFO Test size: 50


In [9]:
c = classification.split.ClusterSpliter("../data/resultsDB_clu.tsv", 5, random_state=experiment.seed)
X_train, X_test = c.train_test_split(data.features)

In [10]:
results, models = training.generate_training_results(X_train, data.label, tune=True, test_data=X_test, fold_strategy=c)

2024/07/27 11:57:37 INFO mlflow.tracking.fluent: Experiment with name 'Classification' does not exist. Creating a new experiment.
27-07-2024 11:57:38 INFO Added metrics for classification: Average precision
27-07-2024 11:57:38 INFO --------------------------------------------------------
27-07-2024 11:57:38 INFO Training classification models
27-07-2024 11:57:38 INFO The models used ['lr', 'knn', 'nb', 'dt', 'svm', 'rbfsvm', 'gpc', 'mlp', 'ridge', 'rf', 'qda', 'ada', 'gbc', 'lda', 'et', 'xgboost', 'lightgbm', 'catboost', 'dummy']
27-07-2024 11:57:38 INFO The number of models used 19
27-07-2024 11:57:38 INFO Time budget is 20 minutes
27-07-2024 11:57:43 INFO Model lr trained in 0.086 minutes
27-07-2024 11:57:48 INFO Model knn trained in 0.073 minutes
27-07-2024 11:57:51 INFO Model nb trained in 0.059 minutes
27-07-2024 11:57:53 INFO Model dt trained in 0.03 minutes
27-07-2024 11:57:55 INFO Model svm trained in 0.038 minutes
27-07-2024 11:57:57 INFO Model rbfsvm trained in 0.032 minutes


In [11]:
results["not_tuned"]["majority"][0]

Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC  \
Split    Fold                                                             
CV-Train Mean    0.7328  0.8059  0.4406  0.8042  0.5669  0.3980  0.4364   
         Std     0.0245  0.0154  0.0661  0.0399  0.0618  0.0640  0.0585   
CV-Val   Mean    0.6670  0.6575  0.3258  0.6967  0.4224  0.2437  0.2816   
         Std     0.1166  0.0963  0.1924  0.2853  0.1949  0.2334  0.2683   

               Average Precision Score  
Split    Fold                           
CV-Train Mean                   0.7658  
         Std                    0.0186  
CV-Val   Mean                   0.6391  
         Std                    0.0856

In [12]:
results["tuned"]["majority"][0]

Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC  \
Split    Fold                                                             
CV-Train Mean    0.7606  0.9524  0.4880  0.8507  0.6177  0.4625  0.5013   
         Std     0.0229  0.0076  0.0685  0.0323  0.0615  0.0630  0.0549   
CV-Val   Mean    0.6670  0.6509  0.3258  0.6967  0.4224  0.2437  0.2816   
         Std     0.1166  0.0805  0.1924  0.2853  0.1949  0.2334  0.2683   

               Average Precision Score  
Split    Fold                           
CV-Train Mean                   0.9087  
         Std                    0.0114  
CV-Val   Mean                   0.6271  
         Std                    0.0930

##### Acces the ClaasificationExperiment from Pycaret and play with teh models if you want

In [ ]:
experiment.pycaret

In [ ]:
test_set_predictions = training.generate_holdout_prediction(models)

In [15]:
training_output = "classification_results"
l = []
for tune_status, result_dict in results.items():
    for key, value in result_dict.items():
        classification.write_results(f"{training_output}/{tune_status}", *value, sheet_name=key)
    classification.write_results(f"{training_output}/{tune_status}", test_set_predictions[tune_status] , sheet_name=f"test_results")

### View in MlFlow

In [19]:
import mlflow

In [21]:
logs= experiment.get_logs()

In [31]:
logs.iloc[0]

run_id                                      3312c5c4cf7b410a9a98642b3111cdc6
experiment_id                                             225057415584816760
status                                                              FINISHED
artifact_uri               file:///g:/Mi%20unidad/BSC/programs/BioML/exam...
start_time                                  2024-03-28 16:10:51.356000+00:00
                                                 ...                        
tags.mlflow.runName                                        Voting Classifier
tags.Run ID                                 3312c5c4cf7b410a9a98642b3111cdc6
tags.USI                                                                ed0c
tags.mlflow.parentRunId                     22708d4abc3d4236aeb2f24b65e1cbd3
tags.mlflow.source.name    c:\Users\ruite\miniforge3\envs\BioML\Lib\site-...
Name: 0, Length: 257, dtype: object

In [27]:
logged_model = 'runs:/a7a0c3f7eea1461f8887070e3a1164c9/model'
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: a7a0c3f7eea1461f8887070e3a1164c9

## Save Model from pre-trained

In [12]:
from BioML.models import save_model

In [13]:
generate = save_model.GenerateModel(training)
final_model = generate.finalize_model(models["tuned"]["majority"])
generate.save_model(final_model, "model_output/majority")

22-06-2024 10:12:05 INFO ----------Finalizing the model by training it with all the data including test set--------------


Transformation Pipeline and Model Successfully Saved


## Save Model from scratch

In [12]:
from BioML.models import save_model

In [17]:
data = save_model.DataParser("../data/esterase_features.xlsx", "../data/esterase_labels.csv", sheets="ch2_20")
experiment = save_model.PycaretInterface("classification", 200, budget_time=20, best_model=3, 
                                  output_path="classification_training", optimize="MCC", experiment_name="generate_model")
classifier = save_model.Classifier(test_size=0.2, optimize="MCC", selected=("qda", "rbfsvm", "ridge"))
training = save_model.Trainer(experiment, classifier, 5, 30)


27-03-2024 14:28:52 INFO ------------------------------------------------------------------------------
27-03-2024 14:28:52 INFO PycaretInterface parameters
27-03-2024 14:28:52 INFO Seed: 200
27-03-2024 14:28:52 INFO Budget time: 20
27-03-2024 14:28:52 INFO The number of models to select: 3
27-03-2024 14:28:52 INFO Output path: classification_training
27-03-2024 14:28:52 INFO ----------------Trainer inputs-------------------------
27-03-2024 14:28:52 INFO Number of kfolds: 5
27-03-2024 14:28:52 INFO Number of iterations: 30


In [19]:
sorted_results, sorted_models, top_params = training.run_training(data.features, data.label)

2024/03/27 14:28:55 INFO mlflow.tracking.fluent: Experiment with name 'generate_model' does not exist. Creating a new experiment.
27-03-2024 14:28:56 INFO --------------------------------------------------------
27-03-2024 14:28:56 INFO Training classification models
27-03-2024 14:28:56 INFO The models used ['lr']
27-03-2024 14:28:56 INFO Time budget is 20 minutes
27-03-2024 14:29:01 INFO Model lr trained in 0.092 minutes
27-03-2024 14:29:01 INFO Training over: Total runtime 0.092 minutes


Model lr trained in 0.092 minutes


In [20]:
generate = save_model.GenerateModel(training)
models =  generate.train_by_strategy(sorted_models, "majority")
final_model = generate.finalize_model(models)
generate.save_model(final_model, "model_output/logistic")

27-03-2024 14:29:02 INFO ----------Finalizing the model by training it with all the data including test set--------------


Transformation Pipeline and Model Successfully Saved


## Prediction

In [14]:
from BioML.models import predict
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [15]:
training_features = "../data/esterase_features.xlsx"
label = "../data/esterase_labels.csv"
test_features = "../data/esterase_features.xlsx"
outlier_train=()
outlier_test=()
sheet_name="ch2_20"
problem="classification"
model_path="model_output/majority"
scaler="zscore"

#### generate the predictions

In [17]:
feature = predict.DataParser(training_features, label, outliers=outlier_train, sheets=sheet_name)
test_features = feature.remove_outliers(feature.read_features(test_features, "ch2_20"), outlier_test)
predictions = predict.predict(test_features, model_path, problem)
predictions

,hydrophobicity_FASG890101.2.residue25,alphaticr.negativecharger.gap5,BHAR880101.lag9_moran,tri_gram_pssm7077,tri_gram_pssm286,tri_gram_pssm466,tri_gram_pssm3493,tri_gram_pssm1298,tri_gram_pssm5728,tri_gram_pssm7468,...,tri_gram_pssm4023,tri_gram_pssm3159,tri_gram_pssm7678,tri_gram_pssm7234,tri_gram_pssm5801,tri_gram_pssm5367,tri_gram_pssm7177,prediction_label,prediction_score_0,prediction_score_1
#,,,,,,,,,,,,,,,,,,,,,
EH1(72),22.857143,0.055016,-0.102575,0.065363,0.832237,0.442684,0.180234,0.089845,0.490688,0.179098,...,0.624236,0.273944,0.460827,0.272559,0.612637,0.401477,0.029163,1,0.0583,0.9417
EH2(71),25.513197,0.053731,-0.068227,0.067360,0.645581,0.449828,0.113174,0.094496,0.332225,0.160173,...,0.627025,0.241681,0.361142,0.294981,0.617432,0.372408,0.028312,1,0.0624,0.9376
EH3(69),27.873564,0.055556,-0.010242,0.021795,0.756387,0.481429,0.207077,0.123859,0.046267,0.153389,...,0.578253,0.229875,0.456541,0.315422,0.460896,0.360463,0.011199,1,0.0625,0.9375
CalB(68),21.806854,0.034921,-0.083474,0.006633,0.097950,0.037538,0.015682,0.006383,0.026272,0.007791,...,0.093919,0.022174,0.039728,0.034692,0.132978,0.032155,0.005397,0,0.5700,0.4300
EH4(67),25.730993,0.053571,-0.103226,0.065890,0.685630,0.458253,0.117385,0.102536,0.358012,0.155480,...,0.620764,0.234235,0.351025,0.277734,0.608569,0.365297,0.027124,1,0.0622,0.9378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EH141(1),26.797386,0.050773,-0.079524,0.006679,0.134293,0.070896,0.014058,0.003704,0.031492,0.015362,...,0.183355,0.036559,0.034975,0.038975,0.141937,0.026144,0.003584,0,0.9108,0.0892
EH142(1),32.676056,0.034384,0.002960,0.001735,0.103232,0.054376,0.016582,0.007587,0.015728,0.009787,...,0.106868,0.027981,0.123676,0.035153,0.083529,0.015107,0.001517,0,0.9268,0.0732
EH143(1),27.921093,0.064319,0.048037,0.005556,0.089063,0.102606,0.022504,0.006974,0.024215,0.031432,...,0.184709,0.029490,0.134652,0.072092,0.177750,0.035943,0.004516,0,0.9130,0.0870


#### Optional if you want to filter predictions based if that test sample is within the applicability domain

Applicability domain compares eucleadian distance between the features from the training and features from the test.   
If the distance is far from a set threshold then that prediction is discarded since it deviates from the samples the model have seen during training

In [41]:
transformed, scaler_dict, test_x = predict.scale(scaler, feature.drop(), test_features)
filtered_pred = predict.domain_filter(predictions, transformed, test_x, 5) # it returns the predictions appended to the features

In [44]:
filtered_pred.shape, predictions.shape

((120, 20), (147, 19))

In [20]:
 # if you don't apply the domain filter, you can just use the predictions but you have to change the index to sample_0, sample_1, etc

col_name = ["prediction_score", "prediction_label", "AD_number"]
predictions = predictions.loc[:, predictions.columns.str.contains("|".join(col_name))] 
predictions["Index"] = [f"sample_{x}" for x, _ in enumerate(predictions.index)]# only keep the columns with the prediction scores

In [21]:
predictions

,prediction_label,prediction_score_0,prediction_score_1,Index
#,,,,
EH1(72),1,0.0583,0.9417,sample_0
EH2(71),1,0.0624,0.9376,sample_1
EH3(69),1,0.0625,0.9375,sample_2
CalB(68),0,0.5700,0.4300,sample_3
EH4(67),1,0.0622,0.9378,sample_4
...,...,...,...,...
EH141(1),0,0.9108,0.0892,sample_142
EH142(1),0,0.9268,0.0732,sample_143
EH143(1),0,0.9130,0.0870,sample_144


### Separate the fasta file into positive or negative

In [34]:
fasta = "../data/whole_sequence.fasta"
res_dir = "prediction_results_domain"

In [35]:
extractor = predict.FastaExtractor(fasta, res_dir)
positive, negative = extractor.separate_negative_positive(predictions)


In [36]:
extractor.extract(positive, negative, positive_fasta=f"positive.fasta", negative_fasta=f"negative.fasta")